# Using Evidently Prompt Registry with Prompt Templates

In this tutorial, we’ll walk through how to use the **Evidently Prompt Registry** to store, version, and reuse prompts.
We’ll also see how to connect it with **LLMJudge** and prompt templates for evaluation.

## Connect to Evidently Cloud
First, import the CloudWorkspace and authenticate with your token.



In [ ]:
from evidently.ui.workspace import CloudWorkspace

In [ ]:
ws = CloudWorkspace("your token")

### Select Project
You need to provide your `org_id` and the project name where you want to store prompts.

In [ ]:
org_id = "your org id"
project = ws.search_project("your project name", org_id=org_id)[0]

## Create or Load a Prompt
You can create a new prompt or load an existing one by name.


In [ ]:
prompt = ws.prompts.get_or_create_prompt(project.id, "my criteria")
prompt.list_versions()

## Add Prompt Versions
Let’s add new versions of the prompt content.
This helps you track changes over time.


In [ ]:
criteria = "aaaa"
prompt.bump_version(criteria)

In [ ]:
prompt.list_versions()

In [ ]:
prompt.get_version().content

In [ ]:
prompt.bump_version("bbbb")

In [ ]:
prompt.get_version("latest").content.as_text()

In [ ]:
prompt.delete_version(prompt.get_version().id)

In [ ]:
prompt.get_version("latest").content.as_text()

## Delete a Prompt Version
You can also remove versions if needed.


In [ ]:
prompt.delete()

In [ ]:
ws.prompts.list_prompts(project.id)

## Define a Judge with Criteria
Now, let’s define a **judge** that evaluates model responses using a template.
We’ll use a binary classification (GOOD / BAD) with simple criteria.


In [ ]:
from evidently.llm.templates import BinaryClassificationPromptTemplate
from evidently.descriptors import LLMJudge

judge = LLMJudge(provider="openai", model="gpt-4o-mini", template=BinaryClassificationPromptTemplate(
    target_category="GOOD",
    non_target_category="BAD",
    criteria="""Classify the model’s response with the following criteria:
Correctness: Is the response factually accurate?
Clarity: Is the response easy to understand?
Relevance: Does it fully address the question?
Output only one rating: good or bad."""
))

## Store the Judge Template in the Prompt Registry
Instead of keeping the template inline, let’s store it in the registry.


In [ ]:
template_prompt = ws.prompts.get_or_create_prompt(project.id, "my template")
template_prompt.bump_version(judge.feature.template)

In [ ]:
template_prompt.list_versions()

## Reuse the Template
You can now load the template from the registry and create a new judge.


In [ ]:
new_judge = LLMJudge(provider="openai",
                 model="gpt-4o-mini",
                 template=template_prompt.get_version().content.template)
new_judge

## Clean Up
Finally, remove the template prompt if you no longer need it.


In [ ]:
template_prompt.delete()